In [1]:
import numpy as np
import pandas as pd
import sys
import pycountry
import swifter
import pickle

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

#version 3.10 needed
print(sys.executable)

/opt/homebrew/opt/python@3.10/bin/python3.10


In [2]:
!conda env list

# conda environments:
#
base                  *  /Users/lukasburtscher/miniconda3
myEnv                    /Users/lukasburtscher/miniconda3/envs/myEnv
tuwnlpie                 /Users/lukasburtscher/miniconda3/envs/tuwnlpie



In [3]:
#!{sys.executable} -m pip install pycountry

In [4]:
df_country = pd.read_csv('data/countrydyadicvars.csv')
print(df_country.shape)
#print(df_country.dtypes)
df_country.head()

(714999, 39)


,cowc_source,cowc_dest,year,cowc_source_yr,cowc_dest_yr,nwg_ec_source,nwg_ec_source_avg,nwg_ec_dest,nwg_ec_dest_avg,btoa,...,TIV_atob_10,IdealPointDistance,colonized,BIT,conflicts_n,migration_stock,rta,sanctionsinplace,CulturalDist_KogutSingh_4,CulturalDist_KogutSingh_6
0,AD,AE,2000,AD_2000,AE_2000,0.292352,0.0,0.762256,0.0,215.0,...,0.0,1.685514,0,0,0,0.0,0,0,NaN,NaN
1,AD,AE,2001,AD_2001,AE_2001,0.313828,0.0,0.747205,0.0,22107.0,...,0.0,1.802059,0,0,0,0.0,0,0,NaN,NaN
2,AD,AE,2002,AD_2002,AE_2002,0.323126,0.0,0.731824,0.0,36238.0,...,0.0,2.016481,0,0,0,0.0,0,0,NaN,NaN
3,AD,AE,2003,AD_2003,AE_2003,0.338390,0.0,0.740421,0.0,562357.0,...,0.0,2.062506,0,0,0,0.0,0,0,NaN,NaN
4,AD,AE,2004,AD_2004,AE_2004,0.349449,0.0,0.766408,0.0,2712.0,...,0.0,1.994187,0,0,0,0.0,0,0,NaN,NaN


In [5]:
#handle missing data in countries
print(df_country.isnull().sum())

#multinoally, has multinational alliances, replace with 0
df_country['multinoally'] = df_country['multinoally'].fillna(0)

#drop bilatnoally_mp, multinoally_mp, CulturalDist_KogutSingh_4 and CulturalDist_KogutSingh_6
df_country = df_country.drop(['bilatnoally_mp','multinoally_mp','CulturalDist_KogutSingh_4','CulturalDist_KogutSingh_6','cowc_source_yr','cowc_dest_yr'],axis=1)
#for the rest dropn the rows
df_country = df_country.dropna()
print(df_country.shape)

cowc_source                    3853
cowc_dest                      3693
year                              0
cowc_source_yr                    0
cowc_dest_yr                      0
nwg_ec_source                     0
nwg_ec_source_avg                 0
nwg_ec_dest                       0
nwg_ec_dest_avg                   0
btoa                              0
atob                              0
btoa_10                           0
atob_10                           0
bilateral_aid_atob                0
bilateral_aid_atob_10             0
aiddata_a_to_b                    0
aiddata_a_to_b_10                 0
budget_source                     0
budget_dest                       0
staff.share.source                0
staff.share.source_avg            0
staff.share.dest                  0
bilatnoally                       0
multinoally                  565690
militarypersonnel_a            3853
militarypersonnel_b            3673
bilatnoally_mp               293980
multinoally_mp              

In [6]:
df_fdi = pd.read_csv('data/fdi.csv')
print(df_fdi.shape)
#print(df_fdi.dtypes)
df_fdi.head()

(176244, 46)


,invest_actual,year,sic,bvdid,cowc3_dest,cowc3_source,foundingyear,isin,ticker,oprevenue,...,gdp.percapita,gdp.annualgrowth,population,fdi.netinflows,natural.resource.rents,patents,polcon3_val,polcon5_val,icrg,polity2
0,NaN,2010,3648,AT9010000310,AGO,AUT,1950,AT0000837307,ZAG,1.129593e+09,...,3122.780766,0.858713,22514275,2.205298e+09,30.804890,0,0.160323,0.160323,59.000000,-2.0
1,NaN,2010,3648,AT9010000310,ALB,AUT,1950,AT0000837307,ZAG,1.129593e+09,...,4114.140114,3.350067,2927519,1.345415e+09,0.606046,0,0.495057,0.495057,68.666667,9.0
2,NaN,2010,3648,AT9010000310,ARE,AUT,1950,AT0000837307,ZAG,1.129593e+09,...,32024.181603,-5.242922,7917368,1.134288e+09,19.235401,0,0.000000,0.000000,78.708333,-8.0
3,NaN,2010,3648,AT9010000310,ARG,AUT,1950,AT0000837307,ZAG,1.129593e+09,...,8225.137176,-5.918525,40482786,4.017159e+09,3.034460,640,0.360682,0.360682,64.875000,8.0
4,NaN,2010,3648,AT9010000310,ARM,AUT,1950,AT0000837307,ZAG,1.129593e+09,...,2994.342544,-14.100000,2888094,7.600407e+08,0.374260,116,0.464302,0.464302,60.625000,5.0


In [7]:
# write first 1000 rows in a csv for developing
#df_fdi.head(1000).to_csv('data/fdi_1000.csv',index=False)
#df_country.head(1000).to_csv('data/countrydyadicvars_1000.csv',index=False)


#handle missing values in FDI frame
print(df_fdi.isnull().sum())

#invest_actual is the label
df_fdi['invest_actual'] = df_fdi['invest_actual'].fillna(0)
#peers number of companies from same home country in destination country
df_fdi['peers'] = df_fdi['peers'].fillna(0)

#A low equity ratio means that the company primarily used debt to acquire assets
#drop features with too much NaNs > 40%
threshold = 0.4
max_nan_count = threshold * len(df_fdi)
# Filter out columns with more than max_nan_count NaN values
df_fdi = df_fdi.dropna(axis=1, thresh=max_nan_count)

#drop uninteresant columns that are not company relevant
drop_cols = ['bvdid','ticker','Administrative',
            'Economic_pooled',
            'Political_pooled','admindistance_KS',
            'ecodistance_KS','peers','gdp.percapita',
            'gdp.annualgrowth','population','fdi.netinflows',
             'natural.resource.rents','patents','polcon3_val',
             'polcon5_val','icrg','polity2']
df_fdi = df_fdi.drop(drop_cols,axis=1)

#drop rows that have NaN
df_fdi = df_fdi.dropna()
print(df_fdi.shape)



#do the same for country DF


invest_actual             174409
year                           0
sic                            0
bvdid                          0
cowc3_dest                     0
cowc3_source                   0
foundingyear                   0
isin                       81085
ticker                     81285
oprevenue                      0
plbeforetax                    0
netincome                    324
cashflow                    4486
totassets                      0
shareholderfunds               0
currentratio                   0
profitmargin                 998
employees                      0
equityratio               106394
intangibleassets               0
Rdexpenses                103186
debt                           0
gearing                        0
ROA_bt                         0
productivity                 108
age                            0
intangibleassets_ratio         0
RD_intensity              103186
dist                           0
Administrative              1597
Cultural_p

# Preprocessing

In [8]:
# preprocess df_country frame
#extract indicator columns and categorical columns
indicator = ['colonized','BIT','conflicts_n','rta','sanctionsinplace','cowc_source','cowc_dest', 'year']
scaler = StandardScaler()
columns_to_scale = [col for col in df_country.columns if col not in indicator]
# Fit and transform the scaler on the selected columns
df_country[columns_to_scale] = scaler.fit_transform(df_country[columns_to_scale])

#export the country scaler
with open('data/model/country-scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print(df_country.dtypes)
df_country.head()


cowc_source                object
cowc_dest                  object
year                        int64
nwg_ec_source             float64
nwg_ec_source_avg         float64
nwg_ec_dest               float64
nwg_ec_dest_avg           float64
btoa                      float64
atob                      float64
btoa_10                   float64
atob_10                   float64
bilateral_aid_atob        float64
bilateral_aid_atob_10     float64
aiddata_a_to_b            float64
aiddata_a_to_b_10         float64
budget_source             float64
budget_dest               float64
staff.share.source        float64
staff.share.source_avg    float64
staff.share.dest          float64
bilatnoally               float64
multinoally               float64
militarypersonnel_a       float64
militarypersonnel_b       float64
TIV_atob                  float64
TIV_atob_10               float64
IdealPointDistance        float64
colonized                   int64
BIT                         int64
conflicts_n   

,cowc_source,cowc_dest,year,nwg_ec_source,nwg_ec_source_avg,nwg_ec_dest,nwg_ec_dest_avg,btoa,atob,btoa_10,...,militarypersonnel_b,TIV_atob,TIV_atob_10,IdealPointDistance,colonized,BIT,conflicts_n,migration_stock,rta,sanctionsinplace
0,AD,AE,2000,-3.634250,-1.074762,-0.195336,-1.075878,-0.079515,-0.075511,-0.076313,...,-0.209116,-0.03277,-0.038924,0.973287,0,0,0,-0.063517,0,0
1,AD,AE,2001,-3.476923,-1.074762,-0.306635,-1.075878,-0.079510,-0.075486,-0.076312,...,-0.209116,-0.03277,-0.038924,1.125310,0,0,0,-0.063517,0,0
2,AD,AE,2002,-3.408811,-1.074762,-0.420375,-1.075878,-0.079507,-0.075430,-0.076311,...,-0.275453,-0.03277,-0.038924,1.405005,0,0,0,-0.063517,0,0
3,AD,AE,2003,-3.296991,-1.074762,-0.356801,-1.075878,-0.079388,-0.075419,-0.076296,...,-0.251084,-0.03277,-0.038924,1.465041,0,0,0,-0.063517,0,0
4,AD,AE,2004,-3.215969,-1.074762,-0.164635,-1.075878,-0.079514,-0.075503,-0.076296,...,-0.252438,-0.03277,-0.038924,1.375925,0,0,0,-0.063517,0,0


In [9]:
#preprocessing FDI data
indicator = ['invest_actual','cowc3_source',
             'cowc3_dest', 'year', 'sic','foundingyear',
            'isin']
scaler = StandardScaler()
columns_to_scale = [col for col in df_fdi.columns if col not in indicator]
# Fit and transform the scaler on the selected columns
df_fdi[columns_to_scale] = scaler.fit_transform(df_fdi[columns_to_scale])

#export the fdi scaler
with open('data/model/fdi-scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

df_fdi['invest_actual']=df_fdi['invest_actual'].astype('int64')

print(df_fdi.dtypes)
df_fdi.head()

invest_actual               int64
year                        int64
sic                         int64
cowc3_dest                 object
cowc3_source               object
foundingyear                int64
isin                       object
oprevenue                 float64
plbeforetax               float64
netincome                 float64
cashflow                  float64
totassets                 float64
shareholderfunds          float64
currentratio              float64
profitmargin              float64
employees                 float64
intangibleassets          float64
Rdexpenses                float64
debt                      float64
gearing                   float64
ROA_bt                    float64
productivity              float64
age                       float64
intangibleassets_ratio    float64
RD_intensity              float64
dist                      float64
dtype: object


,invest_actual,year,sic,cowc3_dest,cowc3_source,foundingyear,isin,oprevenue,plbeforetax,netincome,...,intangibleassets,Rdexpenses,debt,gearing,ROA_bt,productivity,age,intangibleassets_ratio,RD_intensity,dist
0,0,2010,3648,AGO,AUT,1950,AT0000837307,-0.209368,-0.295971,-0.315564,...,-0.22667,-0.166754,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,0.263769
1,0,2010,3648,ALB,AUT,1950,AT0000837307,-0.209368,-0.295971,-0.315564,...,-0.22667,-0.166754,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,-1.187759
2,0,2010,3648,ARE,AUT,1950,AT0000837307,-0.209368,-0.295971,-0.315564,...,-0.22667,-0.166754,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,-0.288970
3,0,2010,3648,ARG,AUT,1950,AT0000837307,-0.209368,-0.295971,-0.315564,...,-0.22667,-0.166754,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,1.697666
4,0,2010,3648,ARM,AUT,1950,AT0000837307,-0.209368,-0.295971,-0.315564,...,-0.22667,-0.166754,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,-0.772389


In [10]:
# ISO 3 to ISO 2
def convert_iso3_to_iso2(iso3):
    country = pycountry.countries.get(alpha_3=iso3)
    if country:
        return country.alpha_2
    else:
        return np.nan
    

df_fdi['cowc_source'] = df_fdi['cowc3_source'].apply(convert_iso3_to_iso2)
df_fdi['cowc_dest'] = df_fdi['cowc3_dest'].apply(convert_iso3_to_iso2)

df_fdi = df_fdi.drop(['cowc3_source','cowc3_dest'],axis=1)

df_fdi.head()

,invest_actual,year,sic,foundingyear,isin,oprevenue,plbeforetax,netincome,cashflow,totassets,...,debt,gearing,ROA_bt,productivity,age,intangibleassets_ratio,RD_intensity,dist,cowc_source,cowc_dest
0,0,2010,3648,1950,AT0000837307,-0.209368,-0.295971,-0.315564,-0.280095,-0.241992,...,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,0.263769,AT,AO
1,0,2010,3648,1950,AT0000837307,-0.209368,-0.295971,-0.315564,-0.280095,-0.241992,...,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,-1.187759,AT,AL
2,0,2010,3648,1950,AT0000837307,-0.209368,-0.295971,-0.315564,-0.280095,-0.241992,...,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,-0.288970,AT,AE
3,0,2010,3648,1950,AT0000837307,-0.209368,-0.295971,-0.315564,-0.280095,-0.241992,...,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,1.697666,AT,AR
4,0,2010,3648,1950,AT0000837307,-0.209368,-0.295971,-0.315564,-0.280095,-0.241992,...,-0.227673,0.195181,-1.382391,-0.056527,0.047903,-0.083666,-0.172084,-0.772389,AT,AM


In [11]:
result = pd.merge(df_fdi, df_country, left_on=['year','cowc_source','cowc_dest'],
                  right_on=['year','cowc_source','cowc_dest'],how='inner')

result.sort_values('year', inplace=True)
scaler = StandardScaler()
columns_to_scale = ['year', 'foundingyear']
scaler = scaler.fit(result[columns_to_scale])
result[columns_to_scale] = scaler.transform(result[columns_to_scale])

# Use pd.get_dummies for one-hot encoding
columns_to_encode = ['sic', 'cowc_source', 'cowc_dest']
result = pd.get_dummies(result, columns=columns_to_encode)

#export the result scaler
with open('data/model/result-scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


print(result.shape)
result.head()

(66013, 363)


,invest_actual,year,foundingyear,isin,oprevenue,plbeforetax,netincome,cashflow,totassets,shareholderfunds,...,cowc_dest_TN,cowc_dest_TR,cowc_dest_TZ,cowc_dest_UA,cowc_dest_UG,cowc_dest_US,cowc_dest_UY,cowc_dest_VN,cowc_dest_ZA,cowc_dest_ZM
14783,0,-4.257649,-1.267398,DK0010207497,-0.033351,-0.083183,-0.22526,-0.135798,-0.036661,-0.018272,...,0,0,0,0,0,0,0,0,0,0
14806,0,-4.257649,-1.267398,DK0010207497,-0.033351,-0.083183,-0.22526,-0.135798,-0.036661,-0.018272,...,0,0,0,0,0,0,0,0,0,0
14807,0,-4.257649,-1.267398,DK0010207497,-0.033351,-0.083183,-0.22526,-0.135798,-0.036661,-0.018272,...,0,0,0,0,0,0,0,0,0,0
14808,0,-4.257649,-1.267398,DK0010207497,-0.033351,-0.083183,-0.22526,-0.135798,-0.036661,-0.018272,...,0,0,0,0,0,0,0,0,0,0
14809,0,-4.257649,-1.267398,DK0010207497,-0.033351,-0.083183,-0.22526,-0.135798,-0.036661,-0.018272,...,0,0,0,0,0,0,0,0,0,0


In [12]:
result.to_csv('data/preprocessed/preprocessed_data.csv',index=False)
df_fdi.to_csv('data/preprocessed/preprocessed_fdi.csv',index=False)
df_country.to_csv('data/preprocessed/preprocessed_country.csv',index=False)
result.head(1000).to_csv('data/preprocessed/preprocessed_data_1000.csv',index=False)